In [1]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv("data_full_anonym.csv")

df.head()

,individual,alternative,choice,officeID,depWeekDay,OD,fAirline,staySaturday,stayDurationMinutes,totalPrice,totalTripDurationMinutes,dtd,nAirlines,nFlights,isContinental,isDomestic,outDepTime_sin,outDepTime_cos,outArrTime_sin,outArrTime_cos
0,91,19,1,1,2,22,36,1.0,18660.0,230.029999,735.0,23.0,1.0,4.0,1.0,0.0,0.737277,-0.675590,-0.854912,0.518773
1,91,20,0,1,2,22,36,1.0,18235.0,230.029999,1160.0,23.0,1.0,4.0,1.0,0.0,0.737277,-0.675590,-0.854912,0.518773
2,91,21,0,1,2,22,30,1.0,17855.0,247.809998,920.0,24.0,1.0,4.0,1.0,0.0,-0.999762,-0.021815,-0.976296,0.216440
3,91,22,0,1,2,22,30,1.0,17855.0,247.809998,1135.0,24.0,1.0,4.0,1.0,0.0,-0.999762,-0.021815,-0.402747,0.915311
4,91,23,0,1,2,22,30,1.0,280.0,247.809998,1135.0,24.0,1.0,4.0,1.0,0.0,-0.999762,-0.021815,-0.976296,0.216440


In [2]:

# Feature engineering
features = ["fAirline", "staySaturday", "stayDurationMinutes", "totalPrice",
    "totalTripDurationMinutes", "dtd", "nAirlines", "nFlights",
    "isContinental", "isDomestic", "outDepTime_sin", "outDepTime_cos",
    "outArrTime_sin", "outArrTime_cos"
]

In [3]:
import statsmodels.api as sm

# Fit multinomial logit model
model = sm.MNLogit(df["choice"], df[features])
result = model.fit(method='newton', maxiter=100, tol=1e-10, disp=1)


Optimization terminated successfully.
         Current function value: 0.121996
         Iterations 11


In [4]:
print(result.params)

                                 0
fAirline                 -0.019647
staySaturday              0.024451
stayDurationMinutes       0.000001
totalPrice               -0.000538
totalTripDurationMinutes -0.004001
dtd                       0.000530
nAirlines                -1.223677
nFlights                 -0.701494
isContinental             2.213399
isDomestic                0.663293
outDepTime_sin            0.257460
outDepTime_cos           -0.114296
outArrTime_sin           -0.763493
outArrTime_cos            0.282299


In [5]:
print(result.summary())

                          MNLogit Regression Results                          
Dep. Variable:                 choice   No. Observations:              1089961
Model:                        MNLogit   Df Residuals:                  1089947
Method:                           MLE   Df Model:                           13
Date:                Wed, 26 Mar 2025   Pseudo R-squ.:                  0.1205
Time:                        13:03:11   Log-Likelihood:            -1.3297e+05
converged:                       True   LL-Null:                   -1.5119e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
                choice=1       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
fAirline                    -0.0196      0.000    -48.326      0.000      -0.020      -0.019
staySaturday                 0.0245      0.015      1.686      0.092      -0.004       0.

In [6]:
predicted_probs = result.predict(df[features])
predicted_probs

,0,1
0,0.988591,0.011409
1,0.997898,0.002102
2,0.996348,0.003652
3,0.998782,0.001218
4,0.998485,0.001515
...,...,...
1089956,0.949128,0.050872
1089957,0.988067,0.011933
1089958,0.956280,0.043720
1089959,0.998655,0.001345


In [7]:
df["predicted_prob"] = predicted_probs.iloc[:, 1]

In [8]:
df["rank"] = df.groupby("individual")["predicted_prob"].rank(ascending=False, method="first")

# Calculate accuracies
chosen_df = df[df["choice"] == 1]
total_chosen = len(chosen_df)

top_1_accuracy = (chosen_df["rank"] == 1).sum() / total_chosen
top_5_accuracy = (chosen_df["rank"] <= 5).sum() / total_chosen

print(f"Top-1 Accuracy: {top_1_accuracy}")
print(f"Top-5 Accuracy: {top_5_accuracy}")

Top-1 Accuracy: 0.19324909428293718
Top-5 Accuracy: 0.5813967188006244
